In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandasai import SmartDataframe
from pandasai.llm.openai import OpenAI
from dotenv import load_dotenv
import os

# load data

In [5]:
DIR_DATASET = os.path.join(os.getcwd(), 'dataset')
DIR_SUBMISSIONS = os.path.join(os.getcwd(), 'submissions')

train = pd.read_csv(os.path.join(DIR_DATASET, 'train.csv'))
test = pd.read_csv(os.path.join(DIR_DATASET, 'test.csv'))

train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Setup of the openAI environment

In [5]:
# Load environment variables from .env file
load_dotenv()
# Access the real API key from the environment
api_key = os.getenv("OPENAI_API_KEY")

# Uses the openai key to start pandasai
llmodel = OpenAI(api_token=api_key)

# EDA on train.csv dataset

## General overview of the training data

In [6]:
# Load the dataset
train_sdf = SmartDataframe(train, config={"llm": llmodel})

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* 75% of the passengers didn't survived
* Most of pasagers are in 3rd class. Less than 25% of passengers are in 1 & 2 class
* Most of passenger are  between 20 - 38 years old
* Most of passengers travel alone
* Half of passangers paid 14 for their ticket and 75% of them less than 31

In [9]:
# Prompt example
train_sdf.chat("tell me the percentage of survived and not surviving passengers using the column 'Survived'")

{'type': 'string', 'value': 'Percentage of survived passengers: 38.38%, Percentage of not surviving passengers: 61.62%.'}


'Percentage of survived passengers: 38.38%, Percentage of not surviving passengers: 61.62%.'

# PClass survivors

In [10]:
train_sdf.chat("Give me the total number of passengers by Pclass organized in descendant order of it")

,Pclass,Total_Passengers
0,3,491
1,2,184
2,1,216


In [11]:
train_sdf.chat("Give me the of survivors (Survived column) by Pclass organized in descending order of it")

,Survived
Pclass,
3,119
2,87
1,136


In [12]:
train_sdf.chat("Give me the ratio between the number of survivors by Pclass and the total of passengers by Pclass. I want the results by descending order of this ratio")

,Pclass,Survivor_Ratio
0,1,0.629630
1,2,0.472826
2,3,0.242363


* Mos of passengers are in 3 class
* 63% of survivors were in 1 class, 47% in 2 class, and 24% in 3 class
* **Pclass** is a good predictor of passengers than survived

# Sex survivors

In [13]:
train_sdf.chat("Give me the total number of passengers by Sex organized in descendant order of it")

,Sex,Total
0,male,577
1,female,314


In [14]:
train_sdf.chat("Give me the number of survivors by Sex organized in descending order of it")

,Sex,Number of Survivors
0,female,233
1,male,109


In [15]:
train_sdf.chat("Give me the survivors rate by Sex organized in descending order of it")

,Sex,Survivor Rate
0,female,0.742038
1,male,0.188908


* There were more male passengers than female passengers
* 74% of female passengers survived and only 19% of males
* **Sex** is a good surviving predictor

# Age survivors

In [16]:
train_sdf.chat("Plot a histogram of passengers by age")

'D:/Dropbox/Programming/Python/machine_learning_tutorials/learning_ml/titanic_survivors_prediction/exports/charts/temp_chart.png'

In [17]:
train_sdf.chat("Plot a histogram of surviving passengers by age")

'D:/Dropbox/Programming/Python/machine_learning_tutorials/learning_ml/titanic_survivors_prediction/exports/charts/temp_chart.png'

In [18]:
train_sdf.chat("Plot an histogram of survivors by Age for men (Sex = male)")

'D:/Dropbox/Programming/Python/machine_learning_tutorials/learning_ml/titanic_survivors_prediction/exports/charts/temp_chart.png'

In [19]:
train_sdf.chat("Plot an histogram of survivors by Age for men (Sex = female)")

'D:/Dropbox/Programming/Python/machine_learning_tutorials/learning_ml/titanic_survivors_prediction/exports/charts/temp_chart.png'

In [20]:
# Create age beans
train_age = train.copy()

train_age['age_bin'] = pd.cut(train['Age'], bins=range(0, 81, 10), include_lowest=True)
train_age

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_bin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"(20.0, 30.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"(30.0, 40.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"(20.0, 30.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"(30.0, 40.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"(30.0, 40.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,"(20.0, 30.0]"
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,"(10.0, 20.0]"
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,NaN
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,"(20.0, 30.0]"


In [21]:
survivors_age = train_age.groupby('age_bin')['Survived'].agg(['sum', 'count'])
survivors_age['survival_rate'] = round(survivors_age['sum'] *100 / survivors_age['count'], 2)
survivors_age

C:\Users\carlo\AppData\Local\Temp\ipykernel_32140\4108385668.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survivors_age = train_age.groupby('age_bin')['Survived'].agg(['sum', 'count'])


,sum,count,survival_rate
age_bin,,,
"(-0.001, 10.0]",38,64,59.38
"(10.0, 20.0]",44,115,38.26
"(20.0, 30.0]",84,230,36.52
"(30.0, 40.0]",69,155,44.52
"(40.0, 50.0]",33,86,38.37
"(50.0, 60.0]",17,42,40.48
"(60.0, 70.0]",4,17,23.53
"(70.0, 80.0]",1,5,20.00


* Child survived more than other passengers independently of their sex (below 10 years old)
* Old people (60+) didn't survived
* TODO: Create a variable to detect child, regular, and old people (60+)
* **Age** might be an interesting variable but not certain about it but transforming this variable into passengers groups: child, regular, and old can increase the its prediction power

# SibSp survivors: Spouse, husband, brother, sister, stepbrother, stepsister

In [22]:
train_sdf.chat("Count the number of passenger with SubSp. Order them by descending order of it")

,count
SibSp,
8,7
5,5
4,18
3,16
2,28
1,209
0,608


In [23]:
# Count survivors by SibSp
survivors_sibsp = train.groupby('SibSp')['Survived'].agg(['count', 'sum'])
survivors_sibsp.columns = ['Total_Passengers', 'Survivors']

# Calculate survival rate
survivors_sibsp['Survival_Rate'] = survivors_sibsp['Survivors'] / survivors_sibsp['Total_Passengers']

# Sort by number of survivors in descending order
survivors_sibsp_sorted = survivors_sibsp.sort_values('Survivors', ascending=False)

# Format survival rate as percentage
survivors_sibsp_sorted['Survival_Rate'] = survivors_sibsp_sorted['Survival_Rate'].map('{:.1%}'.format)

print("Survival Statistics by Number of Siblings/Spouses (SibSp):")
print(survivors_sibsp_sorted)


Survival Statistics by Number of Siblings/Spouses (SibSp):
       Total_Passengers  Survivors Survival_Rate
SibSp                                           
0                   608        210         34.5%
1                   209        112         53.6%
2                    28         13         46.4%
3                    16          4         25.0%
4                    18          3         16.7%
5                     5          0          0.0%
8                     7          0          0.0%


* 🤔Most passengers traveled with their spouse or with their spouse and their brother/sister -> Need to create a synthetic variable to distinguish them

# Parch survivors: mother, father, daughter, son, stepdaughter, stepson

In [24]:
# Count survivors by SibSp
survivors_parch = train.groupby('Parch')['Survived'].agg(['count', 'sum'])
survivors_parch.columns = ['Total_Passengers', 'Survivors']

# Calculate survival rate
survivors_parch['Survival_Rate'] = survivors_parch['Survivors'] / survivors_parch['Total_Passengers']

# Sort by number of survivors in descending order
survivors_parch_sorted = survivors_parch.sort_values('Survivors', ascending=False)

# Format survival rate as percentage
survivors_parch_sorted['Survival_Rate'] = survivors_parch_sorted['Survival_Rate'].map('{:.1%}'.format)

print("Survival Statistics by Number of Siblings/Spouses (Parch):")
print(survivors_parch_sorted)


Survival Statistics by Number of Siblings/Spouses (Parch):
       Total_Passengers  Survivors Survival_Rate
Parch                                           
0                   678        233         34.4%
1                   118         65         55.1%
2                    80         40         50.0%
3                     5          3         60.0%
5                     5          1         20.0%
4                     4          0          0.0%
6                     1          0          0.0%


* 🤔 Most children traveled with one or both parents (Parch = 1, 2)
* 🤔Most parents traveling with their up to 3 children (Parch = 3)
* Most survivors are people traveling with 1 or 2 parents

-> Need to crate a synthetic variable to distinguish them

# Ticket survivors

In [25]:
# Count survivign passengers by Ticket
ticket_survivors = train.groupby('Ticket')['Survived'].agg(['count', 'sum'])
ticket_survivors.columns = ['Total_passengers', 'Survivors']

# Calculate survival rate
ticket_survivors['Survival_Rate'] = ticket_survivors['Survivors'] / ticket_survivors['Total_passengers']
ticket_survivors  = ticket_survivors.sort_values(by=['Survival_Rate'], ascending=False)

# Format survival rate as percentage
ticket_survivors['Survival_Rate'] = ticket_survivors['Survival_Rate'].map('{:.1%}'.format)



print("Survival Statistics by Ticket:")
ticket_survivors

Survival Statistics by Ticket:


,Total_passengers,Survivors,Survival_Rate
Ticket,,,
110152,3,3,100.0%
26360,2,2,100.0%
386525,1,1,100.0%
382651,1,1,100.0%
244373,1,1,100.0%
...,...,...,...
349246,1,0,0.0%
349245,1,0,0.0%
2669,1,0,0.0%


* People with the same ticket number has more tendency to survive
* 🤔 Replace the Ticket variable by a synthetic variable to assign to each ticket number the number of persons associated to the ticket

# Fare survivors

In [28]:
# Count surviving passengers by Fare
fare_survivors = train.groupby('Fare')['Survived'].agg(['count', 'sum'])
fare_survivors.columns = ['Total_passengers', 'Survivors']

# Calculate survival rate
fare_survivors['Survival_Rate'] = fare_survivors['Survivors'] / fare_survivors['Total_passengers']
fare_survivors  = fare_survivors.sort_values(by=['Survival_Rate'], ascending=False)

# Format survival rate as percentage
fare_survivors['Survival_Rate'] = fare_survivors['Survival_Rate'].map('{:.1%}'.format)

print("Survival Statistics by Fare:")
fare_survivors

Survival Statistics by Fare:


,Total_passengers,Survivors,Survival_Rate
Fare,,,
512.3292,3,3,100.0%
57.9792,2,2,100.0%
13.8583,1,1,100.0%
13.7917,1,1,100.0%
13.4167,1,1,100.0%
...,...,...,...
15.5500,1,0,0.0%
47.1000,1,0,0.0%
46.9000,6,0,0.0%


# Cabin survivors

Not considered because there a lot of records missing in the dataset

# Embarqued survivors

In [27]:
# Count surviving passengers by Embarqued (city of embarking)
embarked_survivors = train.groupby('Embarked')['Survived'].agg(['count', 'sum'])
embarked_survivors.columns = ['Total_passengers', 'Survivors']

# Calculate survival rate
embarked_survivors['Survival_Rate'] = embarked_survivors['Survivors'] / embarked_survivors['Total_passengers']
embarked_survivors  = embarked_survivors.sort_values(by=['Survival_Rate'], ascending=False)

# Format survival rate as percentage
embarked_survivors['Survival_Rate'] = embarked_survivors['Survival_Rate'].map('{:.1%}'.format)

print("Survival Statistics by Embarked city (C = Cherbourg, Q = Queenstown, S = Southampton):")
embarked_survivors

Survival Statistics by Embarked city (C = Cherbourg, Q = Queenstown, S = Southampton):


,Total_passengers,Survivors,Survival_Rate
Embarked,,,
C,168,93,55.4%
Q,77,30,39.0%
S,644,217,33.7%


# Conclussion
## Good predictors
* Pclass
* Sex
* Age but transformed/engineered into a categorical variable to separete child, regular, and old passengers (60+)
* SibSp but through a synthetic variable to differentiate the couple traveling with either 0 or 1 children
* Parch but through an synthetic variable to differentiate children traveling with 1 or 2 parents
* Replace the Ticket variable by a synthetic variable to assign to each ticket number the number of persons associated to the ticket -> The synthetic variable can be 'traveling_together' = 1: Passengers traveling with other members of their family 0: Otherwise. Passengers traveling together have the same fare and ticket numbers
* Fare can be used along with the Ticket variable to distinguish passengers travelling together
* People embarked at Cherbourg had the highest survival rate -> Create a dummy variable to distinguish them <br>

**Consider create synthetic variables for:**
* Group passenger by age groups
* Passengers traveling alone: Sibsp + Parch = 1
* Extract title from name

**Further refinement**
* Need to get a look for the variable 'Name': Look at those articles [Feature engineering Name 1](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/) and [Feature engineering Name 2](https://gertlowitz.blogspot.com/2013/06/where-am-i-up-to-with-titanic-competion.html)